In [1]:
import pickle
from pathlib import Path
import pandas as pd

In [2]:
video_root = Path('/root/data/raw/drive_and_act/')
annotation_path = video_root / 'iccv_activities_3s' / 'activities_3s' / 'inner_mirror' / 'midlevel.chunks_90.csv'
raw_ann_df = pd.read_csv(annotation_path)

In [3]:
with open('output/inner_mirror_with_padding/incorrect/pose2D.pkl', 'rb') as f:
    pose2d_list = pickle.load(f)

with open('output/inner_mirror_with_padding/incorrect/frame_info.pkl', 'rb') as f:
    frame_info_list = pickle.load(f)

# with open('output/inner_mirror_with_padding/annotation.pkl', 'rb') as f:
#     annotation_df = pickle.load(f)

In [4]:
frame_info_df = pd.DataFrame(frame_info_list)
pose_2d_df = pd.DataFrame(pose2d_list)

In [5]:
df = frame_info_df.merge(pose_2d_df, on='frame_index')
df = df.reset_index()
df.frame_index = df.index
frame_info_df = df[frame_info_df.columns]
pose_2d_df = df[pose_2d_df.columns]

In [6]:
frame_info_list = frame_info_df.to_dict(orient='records')
pose2d_list = pose_2d_df.to_dict(orient='records')
pickle_path = './output/inner_mirror_with_padding'
frame_info_pickle_path = Path(pickle_path) / "frame_info.pkl"
annotation_pickle_path = Path(pickle_path) / "annotation.pkl"
pose2D_pickle_path = Path(pickle_path) / "pose2D.pkl"

with open(frame_info_pickle_path, "wb") as f:
    pickle.dump(frame_info_list, f)
with open(pose2D_pickle_path, "wb") as f:
    pickle.dump(pose2d_list, f)

In [7]:
annotation_df = pd.DataFrame.from_records(frame_info_list)\
    .groupby(['participant_id', 'video_name', 'annotation_id'])\
    .agg({
        'activity': 'first',
        'frame_index': ['min', 'max']
    }).reset_index()
annotation_df.columns = annotation_df.columns.map(lambda x: '_'.join([str(i) for i in x]))
annotation_df = annotation_df.rename(columns={
    'participant_id_': 'participant_id',
    'video_name_': 'video_name',
    'annotation_id_': 'annotation_id',
    'activity_first': 'activity',
    'annotated_frame_start_first': 'annotated_frame_start',
    'annotated_frame_end_first': 'annotated_frame_end',
    'frame_index_min': 'frame_index_start',
    'frame_index_max': 'frame_index_end'
})
annotation_df['frame_index_end'] = annotation_df['frame_index_end'] + 1
with open(annotation_pickle_path, 'wb') as f:
    pickle.dump(annotation_df, f)

In [8]:
# import pandas as pd
# from pathlib import Path
# pickle_path = './output/inner_mirror_with_padding'
# pose2D_pickle_path = Path(pickle_path) / "pose2D.pkl"
# annotation_pickle_path= Path(pickle_path) / "annotation.pkl"
# annotation_df = pd.DataFrame.from_records(frame_info_list)\
#     .groupby(['participant_id', 'video_name', 'annotation_id'])\
#     .agg({
#         'activity': 'first',
#         'annotated_frame_start': 'first',
#         'annotated_frame_end': 'first',
#         'frame_index': ['min', 'max']
#     }).reset_index()
# annotation_df.columns = annotation_df.columns.map(lambda x: '_'.join([str(i) for i in x]))
# annotation_df = annotation_df.rename(columns={
#     'participant_id_': 'participant_id',
#     'video_name_': 'video_name',
#     'annotation_id_': 'annotation_id',
#     'activity_first': 'activity',
#     'annotated_frame_start_first': 'annotated_frame_start',
#     'annotated_frame_end_first': 'annotated_frame_end',
#     'frame_index_min': 'frame_index_start',
#     'frame_index_max': 'frame_index_end'
# })
# annotation_df['frame_index_end'] = annotation_df['frame_index_end'] + 1
# with open(annotation_pickle_path, 'wb') as f:
#     pickle.dump(annotation_df, f)